In [60]:
import pandas as pd 
import pinecone
import os 
from pinecone import Pinecone,ServerlessSpec
from dotenv import load_dotenv,find_dotenv
from sentence_transformers import SentenceTransformer

In [61]:
file = pd.read_csv('365 Courses data.csv',encoding="ANSI")

In [62]:
file.head()

,course_name,course_slug,course_technology,course_description,course_topic,course_description_short
0,Introduction to Tableau,tableau,tableau,Tableau is now one of the most popular busines...,data visualization,Teaching you how to tell compelling stories wi...
1,The Complete Data Visualization Course with Py...,data-visualization,python,The Data Visualization course is designed for ...,data visualization,Teaching you how to master the art of creating...
2,Introduction to R Programming,introduction-to-r-programming,r,R is one of the best programming languages spe...,programming,"Providing you with the skills to manipulate, a..."
3,Data Preprocessing with NumPy,data-preprocessing-numpy,python,This course is designed to show you how to wor...,data processing,This course will guide you through one of Pyth...
4,Introduction to Data and Data Science,intro-to-data-and-data-science,theory,Working with data is an essential part of main...,machine learning,Introducing you to the field of data science a...


In [63]:
def create_course_description(row):
    return f'''The course name is {row["course_name"]} ,the slug name is {row["course_slug"]} and 
    the technology is {row["course_technology"]} and the course topic is {row["course_topic"]}'''

In [64]:
pd.set_option('display.max_rows',106)
file['coloumn_description_new'] = file.apply(create_course_description,axis=1)
print(file['coloumn_description_new'])

0      The course name is Introduction to Tableau ,th...
1      The course name is The Complete Data Visualiza...
2      The course name is Introduction to R Programmi...
3      The course name is Data Preprocessing with Num...
4      The course name is Introduction to Data and Da...
5      The course name is Data Cleaning and Preproces...
6      The course name is Introduction to Business An...
7      The course name is Data Analysis with Excel Pi...
8      The course name is SQL ,the slug name is sql a...
9      The course name is Credit Risk Modeling in Pyt...
10     The course name is Python Programmer Bootcamp ...
11     The course name is SQL + Tableau + Python ,the...
12     The course name is Introduction to Jupyter ,th...
13     The course name is Statistics ,the slug name i...
14     The course name is Mathematics ,the slug name ...
15     The course name is Introduction to Excel ,the ...
16     The course name is Probability ,the slug name ...
17     The course name is Start

In [65]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [66]:
load_dotenv(find_dotenv(),override=True)

True

In [67]:
api_key = os.environ.get('PINECONE_API_KEY')
env = os.environ.get('PINECONE_ENV')

pc = Pinecone(api_key=api_key,enviorment = env)

In [68]:
#Creating a index 
pc.create_index(
    name='semanticsearchindex',
    dimension= 768,
    metric='cosine',
    spec=ServerlessSpec(
        cloud = 'aws',
        region = 'us-east-1'
   )
)

In [69]:
index = pc.Index('semanticsearchindex')

In [70]:
#Embedding the data 
model = SentenceTransformer('multi-qa-mpnet-base-cos-v1')

In [71]:
def create_embedding(row):
    combined_text = ''.join([str(row[feild] for feild in ['course_description','coloumn_description_new','course_description_short'])])
    embedding = model.encode(combined_text,show_progress_bar=False)
    return embedding

In [72]:
#Creating new Columns 
file['embeddings'] = file.apply(create_embedding,axis =1)

In [73]:
vectors_to_upsert = [(str(row['course_name']),row['embeddings'].tolist())for _,row in file.iterrows()]
index.upsert(vectors=vectors_to_upsert)

print("Data upserted Sucessfully")

Data upserted Sucessfully


In [74]:
#Semantic Search 
query = 'cluestring'
query_embedding = model.encode(query,show_progress_bar=False).tolist()


In [75]:
query_result = index.query(
    vector=[query_embedding],
    top_k=12,
    include_values=True
)    

In [51]:
for match in query_result['matches']:
    if match['score'] >= 0.03:    
        print(f"matched item id -:{(match['id'])}, Score : {match['score']}")

matched item id -:Introduction to R Programming, Score : 0.0349951349
matched item id -:Data Preprocessing with NumPy, Score : 0.0349951349
matched item id -:Introduction to Data and Data Science, Score : 0.0349951349
matched item id -:Data Cleaning and Preprocessing with pandas, Score : 0.0349951349
matched item id -:Introduction to Business Analytics, Score : 0.0349951349
matched item id -:Data Analysis with Excel Pivot Tables, Score : 0.0349951349
matched item id -:SQL, Score : 0.0349951349
matched item id -:Credit Risk Modeling in Python, Score : 0.0349951349
matched item id -:Python Programmer Bootcamp, Score : 0.0349951349
matched item id -:SQL + Tableau + Python, Score : 0.0349951349
matched item id -:Introduction to Tableau, Score : 0.0349951349
matched item id -:The Complete Data Visualization Course with Python, R, Tableau, and Excel, Score : 0.0349951349
